In [1]:
import torch
from torch import nn, optim
from torch.nn.parameter import Parameter
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F
from torch.distributions import Uniform
# from torch.optim.lr_scheduler import ReduceLROnPlateau, ExponentialLR, OneCycleLR

import pytorch_lightning as pl
from pytorch_lightning import Trainer
# from pytorch_lightning.callbacks.early_stopping import EarlyStopping
# from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.loggers import WandbLogger

import torchmetrics

import numpy as np
import matplotlib.pyplot as plt
import random

import wandb

PATH_DATASETS = "."
AVAIL_GPUS = min(1, torch.cuda.device_count())
BATCH_SIZE = 512 if AVAIL_GPUS else 64
# BATCH_SIZE=1

In [2]:
pl.seed_everything(125)

Global seed set to 125


125

In [3]:
NORMALIZE = True
NUM_LAYERS = 2
HIDDEN_DIM = 2
LEARNING_RATE = 1e-2
NUM_ITERS = int(1e5)
RANGE = [5, 10]

## Problem Description

$$a,\, b,\, c,\, d(=-c) \Rightarrow a-c(= a+d),~b+c(=b-d)$$

## Generate Data

In [4]:
class DataWrapper(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [5]:
def degenerate_data(range_feature=[0,1], size=None):
    U = Uniform(*range_feature)
    X = U.sample((size, 3))
    X = torch.column_stack([X, -X[:,2]])
    Y = torch.column_stack([X[:,0] - X[:,2], X[:,1] + X[:,2]])
    ds = DataWrapper(X, Y)
    return ds

In [6]:
ds_test = degenerate_data(range_feature=[0, 1], size=100)

In [7]:
ds_test[0]

(tensor([ 0.7219,  0.3854,  0.7279, -0.7279]), tensor([-0.0060,  1.1132]))

In [8]:
len(ds_test)

100

## Network Setup

In [9]:
class NAC(pl.LightningModule):
    def __init__(self, n_in, n_out):
        super().__init__()
        self.W_hat = Parameter(torch.Tensor(n_out, n_in))
        self.M_hat = Parameter(torch.Tensor(n_out, n_in))
        self.reset_parameters()
        
    def reset_parameters(self):
        nn.init.kaiming_uniform_(self.W_hat)
        nn.init.kaiming_uniform_(self.M_hat)
    
    def forward(self, x):
        weights = torch.tanh(self.W_hat) * torch.sigmoid(self.M_hat)
        return F.linear(x, weights)
    
    def training_step(self, batch, batch_idx):
        X, Y = batch
        
        out = self(X)
        loss = F.mse_loss(out, Y)
        mea = torch.mean(torch.abs(Y - out))
        self.log('train_loss', loss)
        self.log('train_mea', mea)
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        X, Y = batch
        
        out = self(X)
        loss = F.mse_loss(out, Y)
        mea = torch.mean(torch.abs(Y - out))
        self.log('val_loss', loss)
        self.log('val_mea', mea)
        
        return loss
    
    def configure_optimizers(self):
        optimizer = optim.RMSprop(self.parameters(), lr=LEARNING_RATE)
        return optimizer
    
    def prepare_data(self):
        self.ds_train = degenerate_data(range_feature=[0,1], size=500)
        self.ds_val = degenerate_data(range_feature=[0,1], size=100)
        
#     def setup(self, stage=None):
#         if stage == "fit" or stage is None:
#             self.ds_train = DataWrapper(self.X_train, self.y_train)
#             self.ds_val = DataWrapper(self.X_val, self.y_val)
#         if stage == "test" or stage is None:
#             self.ds_test = DataWrapper(self.X_val, self.y_val)
        
    def train_dataloader(self):
        return DataLoader(self.ds_train, batch_size=50)
    
    def val_dataloader(self):
        return DataLoader(self.ds_val, batch_size=10)

In [10]:
model = NAC(
    n_in=4,
    n_out=2
)

wandb_logger = WandbLogger(
    project='NALU_Degenerate_Test'
)

trainer = Trainer(
    logger=wandb_logger,
    max_epochs=200,
    gpus=AVAIL_GPUS,
    enable_progress_bar=False,
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [11]:
model

NAC()

In [12]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type | Params
------------------------------
------------------------------
16        Trainable params
0         Non-trainable params
16        Total params
0.000     Total estimated model params size (MB)
/home/xteca/anaconda3/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning: Checkpoint directory /home/xteca/Documents/Project/Machine_Learning/NALU/None/version_None/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
/home/xteca/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Global seed set to 125
/home/xteca/anaco

In [13]:
wandb.finish()

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_loss,█▅▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_mea,█▆▅▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_loss,█▄▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mea,█▅▄▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,199
train_loss,0.0
train_mea,0.00157
trainer/global_step,1999
val_loss,1e-05


In [14]:
model.W_hat

Parameter containing:
tensor([[ 4.3008,  0.0146, -1.5075,  0.8200],
        [ 0.0091,  3.5475,  0.7256, -0.6659]], requires_grad=True)

In [15]:
model.M_hat

Parameter containing:
tensor([[ 5.8132, -2.4077,  0.8028,  0.2258],
        [-1.6672,  6.4504,  1.7930,  1.4689]], requires_grad=True)

In [16]:
model.eval()

torch.tanh(model.W_hat) * torch.sigmoid(model.M_hat)

tensor([[ 0.9967,  0.0012, -0.6260,  0.3755],
        [ 0.0014,  0.9968,  0.5318, -0.4733]], grad_fn=<MulBackward0>)